In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common import NoSuchElementException


In [27]:
# create a Chrome web driver instance
driver = webdriver.Chrome(service=Service())

# connect to the target page
driver.get(
    "https://www.booking.com/searchresults.en-gb.html?ss=Pila%2C+Valle+d%27Aosta%2C+Italy&efdco=1&label=gen173nr-10CAEoggI46AdIM1gEaLsBiAEBmAEzuAEHyAEM2AED6AEB-AEBiAIBqAIBuAKp5cXHBsACAdICJGY1MjYxNjA3LWI3MzQtNGI4My1hYzBlLWNkZTY0Njk5YzBkZNgCAeACAQ&sid=6c6b02e3cbfefa08662a237131922855&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=index&dest_id=900040971&dest_type=city&ac_position=1&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=440f9e957c14001c&ac_meta=GhA0NDBmOWU5NTdjMTQwMDFjIAEoATICZW46CnBpbGEgYW9zdGFAAEoAUAA%3D&checkin=2026-02-13&checkout=2026-02-17&group_adults=6&no_rooms=1&group_children=0"
)

# scraping logic...

In [28]:
close_items = [
    "button#onetrust-reject-all-handler",
    "button[aria-label='Dismiss sign-in info.']",
]

for i in close_items:
    try:
        # wait up to 20 seconds for the sign-in alert to appear
        close_button = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, i))
        )
        # click the close button
        close_button.click()
    except TimeoutException:
        print("Sign-in modal did not appear, continuing...")

Sign-in modal did not appear, continuing...


In [29]:
def handle_no_such_element_exception(data_extraction_task):
    try:
        return data_extraction_task()
    except NoSuchElementException as e:
        return None

In [ ]:
import time
from selenium.webdriver.common.action_chains import ActionChains

# Scroll to the bottom of the page gradually
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # wait for new results to load
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # reached the bottom
    last_height = new_height

# Now get all property cards
property_items = driver.find_elements(By.CSS_SELECTOR, "[data-testid='property-card']")
items = []

In [ ]:
for property_item in property_items:
    # scraping logic...
    url = handle_no_such_element_exception(
        lambda: property_item.find_element(
            By.CSS_SELECTOR, "a[data-testid='property-card-desktop-single-image']"
        ).get_attribute("href")
    )
    image = handle_no_such_element_exception(
        lambda: property_item.find_element(
            By.CSS_SELECTOR, "img[data-testid='image']"
        ).get_attribute("src")
    )
    title = handle_no_such_element_exception(
        lambda: property_item.find_element(
            By.CSS_SELECTOR, "[data-testid='title']"
        ).text
    )
    address = handle_no_such_element_exception(
        lambda: property_item.find_element(
            By.CSS_SELECTOR, "[data-testid='address']"
        ).text
    )
    distance = handle_no_such_element_exception(
        lambda: property_item.find_element(
            By.CSS_SELECTOR, "[data-testid='distance']"
        ).text
    )
    review_score = None
    review_count = None
    review_text = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid='review-score']").text)
    if review_text is not None:
        # split the review string by n
        parts = review_text.split("n")

    # process each part
    for part in parts:
        part = part.strip()
        # check if this part is a number (potential review score)
        if part.replace(".", "", 1).isdigit():
            review_score = float(part)
        # check if it contains the "reviews" string
        elif "reviews" in part:
            tokens = part.split()
            for token in tokens:
                if token.replace(",", "").isdigit():
                    review_count = int(token.replace(",", ""))
                    break

    
    description = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid='recommended-units']").text)

    price_element = handle_no_such_element_exception(lambda: (property_item.find_element(By.CSS_SELECTOR, "[data-testid='availability-rate-information']")))
    
    original_price = None
    price = None

    if price_element is not None:
        original_price = handle_no_such_element_exception(lambda: (
            price_element.find_element(By.CSS_SELECTOR, "[aria-hidden='true']:not([data-testid])").text.replace(",", "")
        ))
        price = handle_no_such_element_exception(lambda: (
            price_element.find_element(By.CSS_SELECTOR, "[data-testid='price-and-discounted-price']").text.replace(",", "")
        ))
    item = {
    "url": url,
    "image": image,
    "title": title,
    "address": address,
    "distance": distance,
    "review_score": review_score,
    "review_count": review_count,
    "description": description,
    "original_price": original_price,
    "price": price
    }
    items.append(item)

In [31]:
items

[{'url': 'https://www.booking.com/hotel/it/le-volpi-luxury-apartments-gressan.en-gb.html?label=gen173nr-10CAEoggI46AdIM1gEaLsBiAEBmAEzuAEHyAEM2AED6AEB-AEBiAIBqAIBuAKp5cXHBsACAdICJGY1MjYxNjA3LWI3MzQtNGI4My1hYzBlLWNkZTY0Njk5YzBkZNgCAeACAQ&sid=6c6b02e3cbfefa08662a237131922855&aid=304142&ucfs=1&arphpl=1&checkin=2026-02-13&checkout=2026-02-17&dest_id=900040971&dest_type=city&group_adults=6&req_adults=6&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=7f189ea9b8f501cc&srepoch=1760654039&all_sr_blocks=884129601_389062597_0_0_0&highlighted_blocks=884129601_389062597_0_0_0&matching_block_id=884129601_389062597_0_0_0&sr_pri_blocks=884129601_389062597_0_0_0__221920&from=searchresults',
  'image': 'https://cf.bstatic.com/xdata/images/hotel/square600/376778193.webp?k=5f38641e54fd7baa89ff067b1d4fb5375aace84201a839f96bbfc2d3a526359b&o=',
  'title': 'Le Volpi Luxury Apartments',
  'address': 'Gressan',
  'distance': '1.8 km from Pila',
  'review_score': None,
  'rev

In [32]:
driver.quit()

In [34]:
len(items)

25